1. 문서의 내용을 읽는다
2. 문서를 쪼갠다    
    -토큰수 초과로 답변을 생성하지 못할수있꼬
    -문서가 길면(인풋이 길면) 답변 생성이 오래걸림
3.임베딩 -> 벡터 데이터베이스에 저장
4.질문이 있으럗 벡터 데이터베이스에 유사도검색
5.유사도 검색으로 가져온 문서를 LLM에 질문과 같이전달


In [ ]:
%pip install --upgrade --quiet  docx2txt langchain-community

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)

loader = Docx2txtLoader("./tax.docx")
documents_list = loader.load_and_split(text_splitter = text_splitter)





In [15]:
len(documents_list)


193

In [3]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model="text-embedding-3-large")
 








In [33]:
from langchain_chroma import Chroma
#데이터를 처음 저장할때 
#database = Chroma.from_documents(documents=documents_list, embedding=embedding, persist_directory="./chroma_db", collection_name="chroma-tax")
#이미 저장된 데이터를 사용할때
database = Chroma(collection_name="chroma-tax", persist_directory="./chroma_db",embedding_function=embedding)

In [25]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=3)


In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")








In [16]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세 전문가입니다
- [context]를 참고해서 사용자의 질문에 답변해주세요

[context]
{retrieved_docs}

[question]
{query} 
"""


In [26]:
ai_message = llm.invoke(prompt)


In [37]:
ai_message.content

AttributeError: 'dict' object has no attribute 'content'

'연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 몇 가지 중요한 요소들을 고려해야 합니다. 소득세는 과세 대상 금액을 기반으로 하여 점진적으로 증가하는 세율을 적용합니다. 일반적으로 소득세 계산시, 근로소득 공제 및 각종 공제를 반영하여 과세표준을 설정한 후, 해당 과표 구간에 맞는 세율을 적용합니다. \n\n1. **근로소득공제**: 먼저 연봉에서 근로소득공제를 차감해야 합니다. 근로소득공제는 소득 구간에 따라 다르게 적용되며, 예를 들어 5천만원의 연봉인 경우 약 1100만원 정도로 추정할 수 있습니다.\n\n2. **기초공제 및 기타 공제**: 본인 및 부양가족에 대한 기초공제, 연금보험료 공제, 기타 보험료 공제 등이 추가로 반영됩니다. 이러한 개인적인 부분을 모른 상태에서는 대략적인 예측만 가능합니다.\n\n3. **과세표준 구간에 따른 세율 적용**: 소득세율은 과세표준에 따라 다르게 적용되며, 5천만원 수준에서는 기본 세율보다 높은 24% 전후의 세율이 적용될 수 있습니다.\n\n4. **소득세 계산**: 계산된 과세표준에 따라 누진세율을 적용하고, 그 결과로 나온 금액 중에서 기납부 세액(예상 간이세액) 및 세액 공제 등을 차감하여 최종 소득세를 산출하게 됩니다.\n\n따라서 정확한 공제 항목과 각각의 금액이 주어지지 않은 경우 개략적인 소득세 금액을 추정하기 어렵습니다. 대략적인 계산으로는 연봉의 약 5~15% 범위 내에서 소득세가 결정될 수 있으나, 정확한 금액은 개인의 구체적인 공제 항목에 따라 달라집니다. 세무 전문가나 관련 계산 프로그램을 이용하여 상세 계산하는 것을 권장합니다.'


In [1]:
%pip install --upgrade --quiet langchain-community langchain langchain-openai faiss-cpu beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [28]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")


d:\study\inflearn-llm\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [29]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [30]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)






In [34]:
ai_message = qa_chain.invoke({"query": query})


In [39]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천 만원의 직장인의 소득세를 정확하게 계산하려면 현행 세법과 공제 항목 등의 세부사항이 필요합니다. 한국의 소득세는 누진세율을 적용하므로, 기본 공제와 연금보험료 등의 공제 항목에 따라 최종 소득세 부담이 달라질 수 있습니다. 자세한 계산이 필요하다면, 국세청의 소득세 계산기를 이용할 수 있습니다.'}